In [1]:
from ClasificadoresEnsamblados import ClassifGSSC

In [63]:
from gssc.infer import EEGInfer
import mne
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score
import numpy as np
from scipy.io import loadmat

# GSSC PRUEBA

In [3]:
from itertools import chain
from sklearn.metrics import precision_recall_fscore_support
def salida_predicha(hypno):
    hip = []
    for k, duracion in enumerate(hypno['duration']):
        n_veces = int(duracion / 30)
        """
        if n_veces == 0 :
            estado = hypno['trial_type'][k]
            hip.extend([estado] * 1)
        else :
        """
        estado = hypno['trial_type'][k]
        hip.extend([estado] * n_veces)
    
    return hip.copy()

In [4]:
# probando el algoritmo de GSSC datos SC
dic = r'H:\Mi unidad\FormatoBIDS\DataseSleep-Casset\sub-001\ses-001\eeg\sub-001_ses-001_task-sleep_eeg.edf'
hip = r'H:\Mi unidad\FormatoBIDS\DataseSleep-Casset\sub-001\ses-001\eeg\sub-001_ses-001_task-sleep_events.tsv'

dic_dublin =r'H:\Mi unidad\FormatoBIDS\DatasetCollegeDublin\sub-001\ses-001\eeg\sub-001_ses-001_task-sleep_eeg.edf'
hip_dublin = r'H:\Mi unidad\FormatoBIDS\DatasetCollegeDublin\sub-001\ses-001\eeg\sub-001_ses-001_task-sleep_events.tsv'

# leo los datos
raw =  mne.io.read_raw_edf(dic, preload =  True)
anotaciones_real  = pd.read_csv(hip, sep  = '\t')

In [6]:
# obtengo las anotaciones en el formato adecuado
anotaciones_real
Y_real = salida_predicha(anotaciones_real)

np.unique(Y_real)
mapa = {'NREM1' :1, 'NREM2':2, 'NREM3':3, 'REM':4, 'WAKE':0}
y_real = [mapa.get(dato) for dato in Y_real][1:]

In [8]:
raw.info['ch_names']


['EEG Fpz-Cz',
 'EEG Pz-Oz',
 'EOG horizontal',
 'Resp oro-nasal',
 'EMG submental',
 'Temp rectal',
 'Event marker']

In [58]:
######################## usando la funcion de GSSC ################################
dic = {'channels' : { 'eog' : [ 'EOG horizontal'], 'eeg': ['EEG Fpz-Cz','EEG Pz-Oz']}}
raw.set_channel_types({'EOG horizontal': 'eog'})
pesos2, anotaciones2 = ClassifGSSC(raw,dic)

c:\Users\Bruno\OneDrive\Escritorio\Proyecto\.venv\lib\site-packages\gssc\infer.py:117: UserWarning: CUDA was selected but cannot be found on this machine. If you believe this is in error, check your CUDA installation. Otherwise set use_cuda to False.
  warnings.warn("CUDA was selected but cannot be found on this "


Filtering raw data in 1 contiguous segment

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal allpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Filter length: 1 samples (0.010 sec)



c:\Users\Bruno\OneDrive\Escritorio\Proyecto\.venv\lib\site-packages\gssc\infer.py:178: UserWarning: WARNING: GSSC was trained on data with a highpass filter of 0.3Hz. These data have a highpass filter of 0.5Hz
  warnings.warn("WARNING: GSSC was trained on data with a highpass "


Cutting 29.99000000000524 seconds from the back.
Not setting metadata
2649 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 2649 events and 3001 original time points ...
0 bad epochs dropped
Inferring permutation 1 of 5
Inferring permutation 2 of 5
Inferring permutation 3 of 5
Inferring permutation 4 of 5
Inferring permutation 5 of 5


In [10]:
################## Pruebo  GSSC  con lso datos de SC / dubin #####################
Net = EEGInfer()
raw.set_channel_types({'EOG horizontal': 'eog'})

anotaciones,tiempos, pesos = Net.mne_infer(inst = raw, eeg=['EEG Fpz-Cz', 'EEG Pz-Oz'], eog=[ 'EOG horizontal'])

Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 45 samples (0.450 sec)



c:\Users\Bruno\OneDrive\Escritorio\Proyecto\.venv\lib\site-packages\gssc\infer.py:117: UserWarning: CUDA was selected but cannot be found on this machine. If you believe this is in error, check your CUDA installation. Otherwise set use_cuda to False.
  warnings.warn("CUDA was selected but cannot be found on this "
c:\Users\Bruno\OneDrive\Escritorio\Proyecto\.venv\lib\site-packages\gssc\infer.py:178: UserWarning: WARNING: GSSC was trained on data with a highpass filter of 0.3Hz. These data have a highpass filter of 0.5Hz
  warnings.warn("WARNING: GSSC was trained on data with a highpass "


Cutting 29.99000000000524 seconds from the back.
Not setting metadata
2649 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 2649 events and 3001 original time points ...
0 bad epochs dropped
Inferring permutation 1 of 5
Inferring permutation 2 of 5
Inferring permutation 3 of 5
Inferring permutation 4 of 5
Inferring permutation 5 of 5


In [26]:
y_prediccion = list(anotaciones)
accuracy  = accuracy_score(y_real, y_prediccion)       
f1_macro = f1_score(y_real, y_prediccion, average='macro')
F1_X_clase = f1_score(y_real, y_prediccion, average=None)
print(accuracy)
print(f1_macro)
print(F1_X_clase)

0.9592298980747452
0.8246074912887644
[0.99226354 0.45977011 0.86821705 0.88940092 0.91338583]


In [60]:
# probando el algoritmo de GSSC datos agus
raw_=r'H:\.shortcut-targets-by-id\1oz7_KTs_0LBRN7CMTRkLDKYI--PGbP_t\CNTR_VMA_NOCHE-BIDS_copy\sub-174\ses-day1\eeg\sub-174_ses-day1_task-sleep_eeg.edf'
hip =  r'H:\.shortcut-targets-by-id\1epHQ10lpZXvKpjB3tVz3XnAwTgWm38El\SUENIO\sub-174\ses-day1\sleep_staging\sub-174_ses-day1_task-sleep_sleepStages.mat'
raw = mne.io.read_raw(raw_, preload=True)
data  =  loadmat(hip)

Extracting EDF parameters from H:\.shortcut-targets-by-id\1oz7_KTs_0LBRN7CMTRkLDKYI--PGbP_t\CNTR_VMA_NOCHE-BIDS_copy\sub-174\ses-day1\eeg\sub-174_ses-day1_task-sleep_eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 6036399  =      0.000 ... 30181.995 secs...


In [63]:
salida_real = data['stageData'][0][0][6].flatten().tolist()
mapa = {0:0, 1:1, 2:2, 3:3, 5:4, 7:0}
y_real = [mapa.get(dato) for dato in salida_real][1:]

In [66]:
raw.info['ch_names']

['A1-A2',
 'Marca',
 'FC1',
 'FC5',
 'C3',
 'P3',
 'P4',
 'C4',
 'FC6',
 'FC2',
 'EOG2',
 'EOG1',
 'EMG',
 'Fz',
 'Cz',
 'Pz']

In [ ]:
Net = EEGInfer()
raw.set_channel_types({ 'EOG2':'eog','EOG1':'eog'})
anotacion,tiempos, pesos = Net.mne_infer(inst = raw, eeg=['FC1','FC5','C3','P3','P4','C4','FC6','FC2', 'Fz','Cz','Pz'], eog=[  'EOG2','EOG1'])

In [71]:
y_prediccion = list(anotacion)
accuracy  = accuracy_score(y_real, y_prediccion)       
f1_macro = f1_score(y_real, y_prediccion, average='macro')
F1_X_clase = f1_score(y_real, y_prediccion, average=None)
print(accuracy)
print(f1_macro)
print(F1_X_clase)

0.7375745526838966
0.6795509536611137
[0.71428571 0.33093525 0.73056995 0.79843444 0.82352941]


# YASA PRUEBA

In [76]:
import yasa
import pandas as pd

In [ ]:
# probando el algoritmo de GSSC datos SC
dic = r'H:\Mi unidad\FormatoBIDS\DataseSleep-Casset\sub-001\ses-001\eeg\sub-001_ses-001_task-sleep_eeg.edf'
hip = r'H:\Mi unidad\FormatoBIDS\DataseSleep-Casset\sub-001\ses-001\eeg\sub-001_ses-001_task-sleep_events.tsv'

dic_dublin =r'H:\Mi unidad\FormatoBIDS\DatasetCollegeDublin\sub-001\ses-001\eeg\sub-001_ses-001_task-sleep_eeg.edf'
hip_dublin = r'H:\Mi unidad\FormatoBIDS\DatasetCollegeDublin\sub-001\ses-001\eeg\sub-001_ses-001_task-sleep_events.tsv'

# leo los datos
raw =  mne.io.read_raw_edf(dic, preload =  True)
anotaciones_real  = pd.read_csv(hip, sep  = '\t')

In [ ]:
pesos2

In [60]:
raw.info.ch_names

['EEG Fpz-Cz',
 'EEG Pz-Oz',
 'EOG horizontal',
 'Resp oro-nasal',
 'EMG submental',
 'Temp rectal',
 'Event marker']

In [68]:
metadata = {'channels' : { 'eog' : [ 'EOG horizontal'], 'eeg': ['EEG Fpz-Cz','EEG Pz-Oz'], 'emg' : ['EMG submental']}}
raw.set_channel_types({'EOG horizontal': 'eog', 'EMG submental': 'emg'})

<RawEDF | sub-001_ses-001_task-sleep_eeg.edf, 7 x 7950000 (79500.0 s), ~424.6 MB, data loaded>

In [70]:
metadata["channels"]["emg"]

['EMG submental']

In [71]:
sls = yasa.SleepStaging(raw, eeg_name= metadata["channels"]["eeg"][0], eog_name=metadata["channels"]["eog"][0], emg_name=metadata["channels"]["emg"][0])
anotaciones = sls.predict()
pesos = sls.predict_proba()

c:\Users\Bruno\OneDrive\Escritorio\Proyecto\.venv\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.24.2 when using version 0.22.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [78]:
p = pd.DataFrame(np.array(pesos2), columns = ['W', 'N1', 'N2', 'N3', 'R'])

In [91]:
p.loc[1200:1500,p.columns]

,W,N1,N2,N3,R
1200,0.192456,0.252340,98.898338,0.323813,0.333052
1201,0.224862,0.286128,98.649765,0.390891,0.448351
1202,0.380246,0.505126,97.582863,0.594227,0.937537
1203,0.538230,0.734006,96.408501,0.810535,1.508732
1204,0.656792,0.901739,95.493011,0.962787,1.985670
...,...,...,...,...,...
1496,0.014458,0.018048,0.015847,0.012300,99.939346
1497,0.016206,0.022080,0.018221,0.013920,99.929565
1498,0.024456,0.031952,0.025221,0.019001,99.899368
1499,12.087590,19.423548,5.345958,2.953599,60.189301


In [92]:
pesos.loc[1200:1500,p.columns]

,W,N1,N2,N3,R
epoch,,,,,
1200,0.068734,0.152577,0.468513,1.800103e-02,0.292175
1201,0.014500,0.137234,0.720954,4.152526e-03,0.123160
1202,0.021751,0.224097,0.497096,2.323113e-03,0.254733
1203,0.039966,0.352371,0.294551,3.886250e-03,0.309225
1204,0.013526,0.674820,0.123879,2.594242e-03,0.185180
...,...,...,...,...,...
1496,0.387274,0.038922,0.024168,5.103418e-03,0.544533
1497,0.242269,0.103052,0.034059,7.291467e-05,0.620547
1498,0.621802,0.088474,0.086955,3.091296e-04,0.202460
